In [ ]:
!pip uninstall -y google-cloud-datastore
!pip install google-cloud-datastore
!pip install --upgrade pip
!yes | pip uninstall dialogflow
!pip install dialogflow==0.3.0

In [ ]:
from google.cloud import datastore
import dialogflow

client = datastore.Client()

In [ ]:
def addEntitiesToDatastore(datafile, dataStoreKind):
    file = open(datafile,'r')
    for i in file:
        locations_key = client.key(dataStoreKind, i.rstrip("\n\r"))
        locations = datastore.Entity(key=locations_key)
        client.put(locations)

addEntitiesToDatastore('LocationFull.txt','Locations')
addEntitiesToDatastore('MealFull.txt','Meals')

In [ ]:
def addEntityToDialogflow(dataStoreKind, dialogflowEntityName):
  query = client.query(kind=dataStoreKind)
  results = list(query.fetch())

  entity_types_client = dialogflow.EntityTypesClient()


  project_id = !(gcloud config get-value project)

  project_agent_path = entity_types_client.project_agent_path(
          project_id[0])

  for element in entity_types_client.list_entity_types(project_agent_path):
    if (element.display_name == dialogflowEntityName):
      entity_type_path = element.name

  project_id = !(gcloud config get-value project)

  entities = []

  for result in results:
    entity = dialogflow.types.EntityType.Entity()
    #print(entity.value)
    entity.value = result.key.name
    entity.synonyms.append(result.key.name)

    entities.append(entity)

  #print entities

  response = entity_types_client.batch_create_entities(
          entity_type_path, entities)

  #print('Entity created: {}'.format(response))

In [ ]:
addEntityToDialogflow('Locations','Locations')

In [ ]:
addEntityToDialogflow('Meals','Meals')